# Profile performance on RecA example
This Python Jupyter notebook profiles the code on the RecA example data.
The goal is to find slow spots to optimize.

Import necessary Python modules.

In [1]:
import contextlib
import cProfile
import os
import pstats
import tempfile
import warnings

import pandas as pd

import alignparse.minimap2
import alignparse.targets

Suppress warnings that clutter output:

In [2]:
warnings.simplefilter('ignore')

Initialize the `Targets` object:

In [3]:
recA_targetfile = '../notebooks/input_files/recA_amplicon.gb'
recA_parse_specs_file = '../notebooks/input_files/recA_feature_parse_specs.yaml'

targets = alignparse.targets.Targets(
                seqsfile=recA_targetfile,
                feature_parse_specs=recA_parse_specs_file)

Create a `Mapper` to run `minimap2`:

In [4]:
mapper = alignparse.minimap2.Mapper(alignparse.minimap2.OPTIONS_CODON_DMS)

FASTQ file to align and parse:

In [5]:
fastq_files = ['../notebooks/input_files/recA_lib-1_ccs.fastq',
               '../notebooks/input_files/recA_lib-1_ccs.fastq']

Now create the SAM file with the mapper, and then parse it:

In [6]:
with contextlib.ExitStack() as stack:
    # Combine both FASTQ files for slightly bigger query
    queryfile = stack.enter_context(tempfile.NamedTemporaryFile(mode='wt',
                                                                suffix='.fastq'))
    text = []
    for fastq_file in fastq_files:
        with open(fastq_file) as f:
            text.append(f.read().rstrip())
    queryfile.write('\n'.join(text))
    queryfile.flush()
    
    # Create samfile with alignments
    samfile = stack.enter_context(tempfile.NamedTemporaryFile(mode='wt',
                                                              suffix='.sam'))
    targets.align(queryfile=queryfile.name,
                  alignmentfile=samfile.name,
                  mapper=mapper)
    
    # temporary file for profiling stats
    pstats_file = stack.enter_context(tempfile.NamedTemporaryFile(mode='wt'))
    
    # profile the alignment parsing
    cProfile.runctx('targets.parse_alignment(samfile.name)',
                    globals(),
                    locals(),
                    filename=pstats_file.name)

    stats = pstats.Stats(pstats_file.name)

Now print the top most time-intensive functions by total and cumulative time:

In [7]:
topn = 10
for timetype in ['tottime', 'cumtime']:
    print('-' * 20 + f" Top {topn} by {timetype} " + '-' * 20)
    (stats
     .strip_dirs()
     .sort_stats(timetype)
     .print_stats(topn)
     )

-------------------- Top 10 by tottime --------------------
Sun Sep  1 19:28:26 2019    /var/folders/fc/7sw280c13755gkvcsqlhlrfc0000gp/T/tmppjy_r_jc

         110749 function calls (109475 primitive calls) in 0.207 seconds

   Ordered by: internal time
   List reduced from 814 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      354    0.056    0.000    0.082    0.000 cs_tag.py:249(get_accuracy)
     1252    0.032    0.000    0.069    0.000 cs_tag.py:298(extract_cs)
     3081    0.009    0.000    0.009    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      354    0.009    0.000    0.013    0.000 utils.py:15(qvals_to_accuracy)
      200    0.008    0.000    0.165    0.001 targets.py:1094(_parse_single_Alignment)
      188    0.007    0.000    0.007    0.000 libcalignedsegment.pyx:1861(get_aligned_pairs)
6543/6133    0.005    0.000    0.032    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     3200